# Abstractive Summarization

In [1]:
% matplotlib inline
# warning: the below makes unicode the default
from __future__ import unicode_literals 

from __future__ import division

import numpy as np 
import pandas as pd 

import nltk
import inspect

from textacy.vsm import Vectorizer
import textacy.vsm

from nltk.tokenize import TweetTokenizer

from scipy.spatial.distance import cosine

from tqdm import *

import re

import os
import kenlm

import math

Loading the data

In [2]:
cowts_tweets = pd.read_pickle('cowts_nltk_tweets.pkl')
term_matrix = np.load('term_matrix_nltk.npy')
vocab_to_idx = np.load('vocab_to_idx_nltk.npy').item()

content_vocab = list(np.load('content_vocab_nltk.npy'))

Tokenizing it using NLTK's twitter tokenizer

In [3]:
cowts_tweets[0][0]

u": LATEST Nepal's Kantipur TV shows at least 21 bodies lined up on ground after 7.9 earthquake"

In [4]:
tokenizer = TweetTokenizer()

In [5]:
print (tokenizer.tokenize(cowts_tweets[0][0]))

[u':', u'LATEST', u"Nepal's", u'Kantipur', u'TV', u'shows', u'at', u'least', u'21', u'bodies', u'lined', u'up', u'on', u'ground', u'after', u'7.9', u'earthquake']


In [6]:
nltk_tweets = []

for tweet in cowts_tweets[0]:
    nltk_tweets.append(tokenizer.tokenize(tweet))

Given these 59 tweets, which consist of a 1000 word summary of the dataset, I want to generate a paragraph summary from the words. I am going to do this in two steps: 

1. First, I am going to generate a bigram of the words. This will map all of the words in all of the tweets to all the other words they are connected to. 

2. Then, I am going to use the bigram to find an optimal 'text path', which I will solve using Integer Linear Programming. 

Some of the tweets are single word tweets (eg. '`Awful`'). These will not contribute anything to the word graph, so I remove them from this list of chosen tweets. 

In [7]:
nltk_tweets = [tweet for tweet in nltk_tweets if len(tweet) > 1]

## Making a word graph

I want to make a bigram; each node of a bigram is two adjacent words in a tweet. I can then generate sentences by traversing paths (which lead to the same word). 

In [8]:
from nltk.util import bigrams

So for a single tweet, a bigram would look like this: 

In [9]:
(list(bigrams(nltk_tweets[0])))

[(u':', u'LATEST'),
 (u'LATEST', u"Nepal's"),
 (u"Nepal's", u'Kantipur'),
 (u'Kantipur', u'TV'),
 (u'TV', u'shows'),
 (u'shows', u'at'),
 (u'at', u'least'),
 (u'least', u'21'),
 (u'21', u'bodies'),
 (u'bodies', u'lined'),
 (u'lined', u'up'),
 (u'up', u'on'),
 (u'on', u'ground'),
 (u'ground', u'after'),
 (u'after', u'7.9'),
 (u'7.9', u'earthquake')]

I can now construct this for all the tweets, simply by adding their respective bigrams together

In [10]:
all_bigrams = [list(bigrams([token for token in tweets])) for tweets in nltk_tweets]

I take the the starting and end nodes of the bigrams, so that I can generate word paths. 

In [11]:
starting_nodes = [single_bigram[0] for single_bigram in all_bigrams]
end_nodes = [single_bigram[-1] for single_bigram in all_bigrams]

In [12]:
all_bigrams = [node for single_bigram in all_bigrams for node in single_bigram]

In [13]:
all_bigrams = list(set(all_bigrams))

These bigrams themselves are not super useful; what I want to do is compile a list of all the word paths through the bigram. I can do this by using the starts of the tweets as the 'beginnings', and the ends as 'ends'. 

In order to limit the number of word paths, I will limit the path length to between 10 and 15 paths. I'm going to implement a breadth first search to find these word paths.

The first step is to take my bigram list and turn it into a dictionary; this will make it easier to find the paths. 

In [14]:
def make_bigram_graph(all_bigrams, start_node):
    bigram = all_bigrams[:]
    
    '''
    Given a bigram, with a defined start node and defined end nodes, this method
    returns a dictionary which serves as a graph for that bigram 
    '''
    def find_children(bigram, node):
        '''
        Given a node, this method finds all its children 
        '''
        second_word = node[1]
        
        children = [node for node in bigram if node[0] == second_word]
        
        return children
   
    bigram_graph = {}
    # start by adding the start node
    bigram_graph[start_node] = find_children(all_bigrams, start_node)
    bigram.remove(start_node)
    
    nodes_to_check = []
    for i in find_children(bigram, start_node):
        nodes_to_check.append(i)
        
    while nodes_to_check: 
        node = nodes_to_check.pop()
        if node in bigram: 
            bigram_graph[node] = find_children(bigram, node)
            bigram.remove(node)
            for i in find_children(bigram, node):
                nodes_to_check.append(i)
    return bigram_graph


In [15]:
bigram_graph = make_bigram_graph(all_bigrams, starting_nodes[1])

In [16]:
len(bigram_graph)

793

Now that I have this dictionary, I am going to implement a breadth first search to find all possible paths between a start node and an end node. 

In [17]:
def breadth_first_search(bigram_graph, start_node, end_node):
    '''
    This method takes as input a graph, a start node and an end node
    and returns all paths which have a length between 10 and 16
    between the two nodes.
    '''
    graph_to_manipulate = dict(bigram_graph)
    
    queue = []
    paths_to_return = []
    queue.append([start_node])
    
    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        # get the last node from the path
        node = path[-1]
        # path found
        if node == end_node:
            if (len(path) < 16) and (len(path) > 10): #limit path length 
                paths_to_return.append(path)
        # enumerate all adjacent nodes, construct a new path and push it into the queue
        for adjacent in graph_to_manipulate.get(node, []):
            new_path = list(path)
            new_path.append(adjacent)
            queue.append(new_path)
        if node in graph_to_manipulate: 
            del graph_to_manipulate[node] # prevents circular references

    return paths_to_return

In [18]:
path = breadth_first_search(bigram_graph, starting_nodes[1], end_nodes[2])

Lets see the first path this produces

In [19]:
path[0]

[(u'MEA', u'opens'),
 (u'opens', u'24hr'),
 (u'24hr', u'Control'),
 (u'Control', u'Room'),
 (u'Room', u'in'),
 (u'in', u'Kathmandu'),
 (u'Kathmandu', u':'),
 (u':', u'Please'),
 (u'Please', u'Retweet'),
 (u'Retweet', u'.'),
 (u'.', u'"')]

This is a word path. Success! 

Now, I just need to repeat this exercise for every starting node, mapping to every end node, to collect a 'total word path' document. 

In [20]:
bigram_paths = []

for single_start_node in tqdm(starting_nodes): 
    bigram_graph = make_bigram_graph(all_bigrams, single_start_node)
    for single_end_node in end_nodes:
        possible_paths = breadth_first_search(bigram_graph, single_start_node, single_end_node)
        for path in possible_paths: 
            bigram_paths.append(path)

100%|██████████| 61/61 [00:24<00:00,  2.54it/s]


In [21]:
len(bigram_paths)

4794

Adding the original tweets to the possible word paths

In [22]:
for tweet in nltk_tweets: 
    bigram_paths.append(list(bigrams([token for token in tweets])))

In [23]:
bigram_paths[4795]

[(u'At', u'least'),
 (u'least', u'two'),
 (u'two', u'dead'),
 (u'dead', u'after'),
 (u'after', u'Nepal'),
 (u'Nepal', u'quake')]

Finally, I want to turn the tweets from bigrams to actual sentences (or at least, lists of unicode). 

In [24]:
def make_list(bigram_path):
    '''
    This method takes a bigram path (eg. [(u'hello', u'world'), (u'world', u'!')]) and returns 
    a list of unicode (eg [u'hello', u'world', u'!')
    '''
    unicode_list = []
    unicode_list.append(bigram_path[0][0])
    unicode_list.append(bigram_path[0][1])
    
    for bigram in bigram_path[1:]:
        unicode_list.append(bigram[1])
    
    return unicode_list

In [25]:
make_list(bigram_paths[4033])

[u'Nepal',
 u'quake',
 u'-',
 u'Buildings',
 u'collapse',
 u'in',
 u'Delhi',
 u'for',
 u'queries',
 u'regarding',
 u'tragic',
 u'Earthquake',
 u'Rocks',
 u'Nepal',
 u'victims']

In [26]:
word_paths = []
for path in tqdm(bigram_paths): 
    word_paths.append(make_list(path))

100%|██████████| 4855/4855 [00:00<00:00, 151368.83it/s]


Given all these paths, I want to find the best ones. 

## COntent Words Based ABstractive Summarization (COWABS) 

One of the constraints in COWABS requires me to make a trigram language model of all the tweets in my word path. This essentially determines the probability of what the next word in the word path will be, using the common sequences of all the words in **some corpus**. 

Basically, my trigram language model will look at sequences of 3 words in the tweet paths, and ask 'does this sequence of three words occur commonly in a corpus of normal readable english? If so, its probably a readable sequence (and has a high probability of occuring in a normal english sentence). 

I now need to create this model, using a corpus of english. 


### Training a Trigram language model

As per [Rudra et al](http://dl.acm.org/citation.cfm?id=2914600) (different paper from the last notebook), I want to maximize
\begin{equation}
\sum_{i=1}^{n} LQ(i)\cdot I(i) \cdot x_{i} + \sum_{j=1}^{m} y_{j}
\end{equation}
where x are the paths chosen, y are the content words chosen, I(i) describes the Informativeness of some word path i and LQ(i) describes the Linguistic Quality Score of some word path i. 

I will need to define the Informativeness and Linguistic Quality Scores quantitatively: 

**Informativeness** is the cosine distance between each word-path and the mean tf-idf vector. 

**Linguistic Quality Score** assigns probabilities to the occurences of words, with more probable words getting higher scores, such that 
\begin{equation}
LQ(s_{i}) = \frac{1}{(1 - ll(w_1, w_2, ... , w_q)}
\end{equation}
where
\begin{equation}
ll(w_{1}, w_{2}, ... , w_{q}) = \frac{1}{L} log_{2} \prod_{t=3}^q P(w_{t}|w_{t-1}w_{t-2})
\end{equation}
The point of this constraint is to weight more probably sequences of words more highly, therefore favouring more 'realistic' sentences. 

As before, the word paths will be subject to the same constraints as the tweets; all content words in a word path must be selected, and if a content word is selected, so must a word path containing it. 

I'm going to start by defining some methods, which will make all these equations easier to define. 

Starting with informativeness, which is quite a bit easier to define: 

In [27]:
def informativeness(word_path):
    '''
    This method returns the cosine difference between
    a tweet path and the mean of the tf-idf term matrix
    
    Input = word path (as a unicode list)
    Ouptut = cosine difference (scalar value)
    '''
    tfidf_mean = np.mean(term_matrix, axis = 0)
    
    # First, I need to construct the tf-idf vector
    tfidf_path = np.zeros(len(tfidf_mean))
    
    for word in word_path: 
        word_idx = vocab_to_idx[word]
        tfidf_path[word_idx] = np.max(term_matrix[:,word_idx])
   
    cosine_difference = cosine(tfidf_mean, tfidf_path)
    return cosine_difference

In [28]:
informativeness(word_paths[1000])

0.64559910081477889

For the linguistic quality score, I am going to be using [kenlm](http://kheafield.com/code/kenlm/) (specifically its python implementation), which uses the trigram language model I have created above to calculate the linguistic quality for me. 

In [29]:
kenlm_model = kenlm.Model('coca.arpa')

Note: the kenlm model takes as input a string, not a list of unicode, so I need to turn the word path into a string sentence before I can pass it to the kenlm model to get its score. 

Since this method depends on the summary length, I will define the summary length `L` here. 

In [30]:
L = 150

Now, I can define the method: 

In [31]:
def linguistic_quality(word_path):
    '''
    This method takes a word path, and returns a linguistic quality score 
    '''
    path_string = str(" ").join([token.encode('ascii', 'ignore') for token in word_path])
    
    ll_score = math.log(10**kenlm_model.score(path_string, bos = True, eos = True), 2)/L
    
    # scale by 100 so this meaningfully impacts the Integer Programming Solution
    return 1/(1-ll_score)

I may be picking the best of a bad bunch here. 

The constraints are actually the same as for the COWTS model:

1. 
\begin{equation}
\sum_{i=1}^{n} x_{i} \cdot Length(i) \leq L
\end{equation}
I want the total length of all the selected word paths to be less than some value L, which will be the length of my summary, L. I can vary L depending on how long I want my summary to be. 

2. 
\begin{equation}
\sum_{i \in T_{j}} x_{i} \geq y_{j}, j = [1,...,m]
\end{equation}
If I pick some content word $y_{j}$ (out of my $m$ possible content words) , then I want to have at least one path from the set of word paths which contain that content word, $T_{j}$. 

3. 
\begin{equation}
\sum_{j \in C_{i}} y_{j} \leq |C_{i}| \times x_{i}, i = [1,...,n]
\end{equation}
If I pick some path i (out of my $n$ possible paths) , then all the content words in that path $C_{i}$ are also selected. 

Let's begin the ILP step, once again using  [PyMathProg](http://pymprog.sourceforge.net/index.html).

In [32]:
from pymprog import *

In [33]:
begin('COWABS')

model('COWABS') is the default model.

In [34]:
# Defining my first variable, x 
# This defines whether or not a word path is selected
x = var(str('x'), len(word_paths), bool)

In [35]:
# Also defining the second variable, which defines
# whether or not a content word is chosen
y = var(str('y'), len(content_vocab), bool)

Now that I have defined my variables, I can define my equation to maximize: 
\begin{equation}
\sum_{i=1}^{n} LQ(i)\cdot I(i) \cdot x_{i} + \sum_{j=1}^{m} y_{j}
\end{equation}

In [36]:
maximize(sum([linguistic_quality(word_paths[i])*informativeness(word_paths[i])*x[i] for i in range(len(x))]) + 
         sum(y));

Now, I can define my constraints. First, 
\begin{equation}
\sum_{i=1}^{n} x_{i} \cdot Length(i) \leq L
\end{equation}

In [37]:
## Maximum length of the entire tweet summary L has been defined above
## when finding the linguistic quality score

# hiding the output of this line since its a very long sum 
sum([x[i]*len(word_paths[i]) for i in range(len(x))]) <= L;

As for COWTS, I define two helper methods for the next two constrains. 

Since I don't have a term matrix, they need to be slighly rewritten. 

In [38]:
def content_words(i):
    '''Given a word path index i (for x[i]), this method will return the indices of the words in the 
    content_vocab[] array
    Note: these indices are the same as for the y variable
    '''
    path = word_paths[i]
    content_indices = []
    
    for word in path:
        if word in content_vocab:
            content_indices.append(content_vocab.index(word))
    return content_indices

In [39]:
def paths_with_content_words(j):
    '''Given the index j of some content word (for content_vocab[j] or y[j])
    this method will return the indices of all tweets which contain this content word
    '''
    content_word = content_vocab[j]
    
    indices = []
    
    for i in range(len(word_paths)):
        if content_word in word_paths[i]:
            indices.append(i)
    
    return indices

I can now define the second constraint: 
\begin{equation}
\sum_{i \in T_{j}} x_{i} \geq y_{j}, j = [1,...,m]
\end{equation}

In [40]:
for j in range(len(y)):
    sum([x[i] for i in paths_with_content_words(j)])>= y[j]

And the third constraint:
\begin{equation}
\sum_{j \in C_{i}} y_{j} \leq |C_{i}| \times x_{i}, i = [1,...,n]
\end{equation}

In [41]:
for i in range(len(x)):
    sum(y[j] for j in content_words(i)) >= len(content_words(i))*x[i]

In [42]:
solve()

'The LP problem instance has been successfully solved. (This code\ndoes {\\it not} necessarily mean that the solver has found optimal\nsolution. It only means that the solution process was successful.) \nThe MIP problem instance has been successfully solved. (This code\ndoes {\\it not} necessarily mean that the solver has found optimal\nsolution. It only means that the solution process was successful.)'

In [43]:
result_x =  [value.primal for value in x]
result_y = [value.primal for value in y]

In [44]:
end()

model('COWABS') is not the default model.

In [45]:
chosen_paths = np.nonzero(result_x)
chosen_words = np.nonzero(result_y)

In [46]:
for i in chosen_paths[0]:
    print ('--------------')
    print str(" ").join([token.encode('ascii', 'ignore') for token in word_paths[i]])

--------------
: LATEST Nepal's Kantipur TV shows at Ahmedabad from Kathmandu RestlessEarth GeographyNow
--------------
MEA opens 24hr Control Room in Nepal 20 00 29 UTC quake
--------------
: EarthquakeInNepal Helpline Numbers of Lamjung , Purvanchal & Kochi too !
--------------
: Warning India Bangladesh Pakistan Afganistan Bhutan Nepal Earthquake Rocks Nepal BBC
--------------
Dharahara also called Bhimsen Tower , 2 at 09:30 UTC , 9851135141
--------------
( Houston _0998 ) Avalanche Sweeps Everest in Nepal - New York Times
--------------
Kathmandu's Darbar Square after 7.9 magnitude in Tibet Nepalquake Embedded image permalink
--------------
5.00 earthquake Kathmandu Ambulance and 11 2301 2113 011 2301 4104 011 2301 7905
--------------
Update on 4/25 / 2015 06:37 UTC , Katmandu - Fox News
--------------
: 09771 4261945 / 15 @ 9:30 : Nepal AssociatedPress Associated Press news
--------------
Bravo Google You are faster than 300 people within 100km . 9 - France 24
--------------
: Pa